In [21]:
# converter of geochemical data for "endmembers" of specific to GHG rocks
# author:  thomas kosciuch 
# contact: thomas@kosciuch.ca
# affilite: University of Toronto - Earth Sciecnes


# 1. Directory Structure
#
#   python code
#           /XSLX/
#               xlsx files
# 

import pandas as pd
from os import listdir
import mysql.connector
import sqlite3

import MySQLdb

In [22]:

################################
# finds files in source folder #
folder = "XLSX"                #
files = listdir(folder)        #
path = folder + "/" + files[0] #
################################

In [23]:
file = pd.ExcelFile(path)
sheets = file.sheet_names
sheet_no = len(sheets)

In [24]:
# for each sheet" 
# file.header

sheet_no

3

In [28]:
import sys

con = None

try:
    con = sqlite3.connect('test.db')

    cur = con.cursor()
    cur.execute('SELECT SQLITE_VERSION()')

    data = cur.fetchone()

    print "SQLite version: {}".format(data)

except sqlite3.Error, e:

    print "Error {}:".format(e.args[0])
    sys.exit(1)

finally:

    if con:
        con.close()  
                     

SyntaxError: invalid syntax (<ipython-input-28-9cb66b75d9f3>, line 13)

In [6]:
# 1. create sql struct per file

def create_connection(db_file):
    """ create a database connection to a SQLite database """
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
    finally:
        conn.close()


In [7]:
conn = sqlite3.connect('mineralEndmembers.db')
c = conn.cursor()
# Create table
c.execute('''CREATE TABLE stocks
             (date text, trans text, symbol text, qty real, price real)''')

# Insert a row of data
c.execute("INSERT INTO stocks VALUES ('2006-01-05','BUY','RHAT',100,35.14)")

# Save (commit) the changes
conn.commit()
conn.close()

In [8]:
variable = ["a", "b"]
len(variable)

2

In [15]:
# f= open("guru99.txt","w+")

TypeError: 'int' object is not iterable

a
